In [35]:
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm.auto import tqdm
from scipy.io import loadmat

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pipeline2 import load_normals, train_cvae, get_errors, find_file, get_anomalies, evaluate_all, find_data_subfolder

In [86]:
PTB_DIR = find_data_subfolder('ptb-xl/1.0.3')
CHAP_DIR = find_data_subfolder('ChapmanShaoxing')

In [5]:
# 1) Carga normales y entrena
normals = load_normals()


In [6]:
model, val_loader, device = train_cvae(normals)

c:\Users\anapt\Repositorios\TP-final_ML\TP-final_ML\ecg_project\pipeline2.py:66: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  train_loader = DataLoader(TensorDataset(torch.tensor(train_set)), batch_size=BATCH, shuffle=True)


Train Ep1/18:   0%|          | 0/293 [00:00<?, ?it/s]

Epoch 1: Train Loss=0.9993 — Val MAE=0.6029


Train Ep2/18:   0%|          | 0/293 [00:00<?, ?it/s]

Epoch 2: Train Loss=0.9911 — Val MAE=0.6029


...

Epoch 16: Train Loss=0.9911 — Val MAE=0.6029


Train Ep17/18:   0%|          | 0/293 [00:00<?, ?it/s]

Epoch 17: Train Loss=0.9911 — Val MAE=0.6029


Train Ep18/18:   0%|          | 0/293 [00:00<?, ?it/s]

Epoch 18: Train Loss=0.9911 — Val MAE=0.6029


In [7]:
# 2) Errores sanos
healthy_errors = get_errors(model, val_loader, device)


In [8]:
# 3) Anómalos
df_meta = pd.read_csv(find_file(PTB_DIR, 'ptbxl_database.csv'))
ptb_errors, chap_errors = get_anomalies(model, df_meta, device)


Anómalos PTB-XL: 21799
Anómalos Chapman: 10247


In [9]:
# 4) Evaluación final
evaluate_all(healthy_errors, ptb_errors, chap_errors)

ROC AUC: 0.473
PR  AUC: 0.933
Best threshold (Youden): 0.7031
Metrics saved to metrics_combined_lead_II.json


Improved

In [103]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM


from preprocess import (
    load_sanos,
    load_ptbxl_anomalies,
    load_chapman_anomalies,
    load_mitbih
)

In [81]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [83]:
def find_file(root, filename):
    """
    Busca recursivamente `filename` bajo `root` y devuelve la ruta completa.
    """
    for r, _, files in os.walk(root):
        if filename in files:
            return os.path.join(r, filename)
    return None


In [ ]:
# 1) Cargar datos
normals = load_sanos() 

In [87]:
# anomalías PTB y Chapman
from preprocess import load_ptbxl, pd


ptb_df = pd.read_csv(find_file(PTB_DIR, 'ptbxl_database.csv'))
# ptb_df = pd.read_csv(os.path.join(os.getcwd(), 'ptbxl_database.csv'))
anom_ptb = load_ptbxl_anomalies(PTB_DIR, ptb_df)
# anom_ptb = load_ptbxl_anomalies(os.getcwd(), ptb_df)
anom_chap = load_chapman_anomalies(CHAP_DIR)

# anom_mit = load_mitbih(os.getcwd())
anomalies = np.concatenate([anom_ptb, anom_chap], axis=0)

In [122]:
# 2) Preparar DataLoaders
X_norm = torch.tensor(normals[:,1:2,:]).unsqueeze(1)  # lead II
X_ano  = torch.tensor(anomalies[:,1:2,:]).unsqueeze(1)
y_norm = np.zeros(len(X_norm))
y_ano  = np.ones(len(X_ano))

dataset_norm = TensorDataset(X_norm, torch.tensor(y_norm))
dataset_ano  = TensorDataset(X_ano,  torch.tensor(y_ano))
# split train/val/test: 70/10/20 para sanos, usar mismo tamaño para anómalos en test
n = len(dataset_norm)
i1, i2 = int(0.7*n), int(0.8*n)
ds_train = torch.utils.data.Subset(dataset_norm, range(0,i1))
ds_val   = torch.utils.data.Subset(dataset_norm, range(i1,i2))
ds_test  = torch.utils.data.Subset(dataset_norm, range(i2,n))
# añadir anomalías SOLO en test para evaluar
ds_test = torch.utils.data.ConcatDataset([ds_test,
                                            torch.utils.data.Subset(dataset_ano, range(0, len(ds_test)))])
# DataLoaders
loader_train = DataLoader(ds_train, batch_size=32, shuffle=True)
loader_val   = DataLoader(ds_val,   batch_size=32, shuffle=False)
loader_test  = DataLoader(ds_test,  batch_size=32, shuffle=False)

In [123]:
from pipeline_improved import VAE1D, beta_cyclic, compute_anomaly_scores, fit_mahalanobis, mahalanobis_distance, grid_search_alpha

In [151]:

# 3) Modelo y optimizador
model = VAE1D(input_ch=1, latent_dim=32, seq_len=2048).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [152]:
from torch.utils.data import DataLoader, TensorDataset, Subset, ConcatDataset

In [153]:
# 2) Preparar TensorDataset de lead II
X_norm = torch.tensor(normals[:,1:2,:], dtype=torch.float32)
y_norm = torch.zeros(len(X_norm), dtype=torch.long)
X_ano  = torch.tensor(anomalies[:,1:2,:], dtype=torch.float32)
y_ano  = torch.ones(len(X_ano), dtype=torch.long)

print(f"DEBUG: X_norm shape: {X_norm.shape}, X_ano shape: {X_ano.shape}")

ds_norm = TensorDataset(X_norm, y_norm)
ds_ano  = TensorDataset(X_ano,  y_ano)

# Splits sanos 70/10/20
n = len(ds_norm)
i1, i2 = int(0.7*n), int(0.8*n)
ds_train = Subset(ds_norm, range(0,i1))
ds_val   = Subset(ds_norm, range(i1,i2))
ds_testN = Subset(ds_norm, range(i2,n))

# DEBUG: longitudes de los splits
print(f"DEBUG: len train: {len(ds_train)}, len val: {len(ds_val)}, len testN: {len(ds_testN)}")
n = len(ds_norm)
i1, i2 = int(0.7*n), int(0.8*n)
ds_train = Subset(ds_norm, range(i1))
ds_val   = Subset(ds_norm, range(i1, i2))
ds_testN = Subset(ds_norm, range(i2, n))

# Anomalías igual tamaño para test
ds_testA = Subset(ds_ano, range(len(ds_testN)))
ds_test  = ConcatDataset([ds_testN, ds_testA])

loader_train = DataLoader(ds_train, batch_size=32, shuffle=True)
loader_val   = DataLoader(ds_val,   batch_size=32, shuffle=False)
loader_test  = DataLoader(ds_test,  batch_size=32, shuffle=False)


DEBUG: X_norm shape: torch.Size([11716, 0, 2048]), X_ano shape: torch.Size([20330, 0, 2048])
DEBUG: len train: 8201, len val: 1171, len testN: 2344


In [169]:
# 2) Preparar TensorDataset de lead II
X_norm = torch.tensor(normals, dtype=torch.float32)  # normals shape already (N,1,2048)
y_norm = torch.zeros(len(X_norm), dtype=torch.long)
X_ano  = torch.tensor(anomalies, dtype=torch.float32)  # anomalies shape (M,1,2048)
y_ano  = torch.ones(len(X_ano), dtype=torch.long)

print(f"DEBUG: X_norm shape: {X_norm.shape}, X_ano shape: {X_ano.shape}")

ds_norm = TensorDataset(X_norm, y_norm)
ds_ano  = TensorDataset(X_ano,  y_ano)

# Splits sanos 70/10/20
n = len(ds_norm)
i1, i2 = int(0.7*n), int(0.8*n)
ds_train = Subset(ds_norm, range(0, i1))
ds_val   = Subset(ds_norm, range(i1, i2))
ds_testN = Subset(ds_norm, range(i2, n))

# DEBUG: longitudes de los splits
print(f"DEBUG: len train: {len(ds_train)}, len val: {len(ds_val)}, len testN: {len(ds_testN)}")

# Anomalías igual tamaño para validación y test
ds_valA = Subset(ds_ano, range(len(ds_val)))
ds_testA = Subset(ds_ano, range(len(ds_testN)))

# Combina normales y anomalías en val y test
ds_val_all  = ConcatDataset([ds_val,  ds_valA])
ds_test_all = ConcatDataset([ds_testN, ds_testA])

loader_train = DataLoader(ds_train,    batch_size=32, shuffle=True)
loader_val   = DataLoader(ds_val_all,  batch_size=32, shuffle=False)
loader_test  = DataLoader(ds_test_all, batch_size=32, shuffle=False)
ds_testA = Subset(ds_ano, range(len(ds_testN)))
ds_test  = ConcatDataset([ds_testN, ds_testA])

loader_train = DataLoader(ds_train, batch_size=32, shuffle=True)
loader_val   = DataLoader(ds_val,   batch_size=32, shuffle=False)
loader_test  = DataLoader(ds_test,  batch_size=32, shuffle=False)

# 3) Modelo y optimizador
model = VAE1D(input_ch=1, latent_dim=64, seq_len=2048).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# 4) Entrenamiento
# Ajustes: LR reducido y gradient clipping para estabilizar
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 20
for ep in range(1, epochs+1):
    model.train()
    total_loss = 0
    for x, _ in loader_train:
        x = x.to(device)
        rec, mu, logvar = model(x)
        recon_loss = torch.mean((rec - x)**2)
        kl_loss = -0.5 * torch.sum(1 + logvar - mu**2 - torch.exp(logvar)) / x.size(0)
        beta = beta_cyclic(ep)
        loss = recon_loss + beta * kl_loss
        optimizer.zero_grad()
        loss.backward()
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()
    print(f"Ep {ep}/{epochs} - Loss: {total_loss/len(loader_train):.4f} - β: {beta:.2f}")


DEBUG: X_norm shape: torch.Size([11716, 1, 2048]), X_ano shape: torch.Size([20330, 1, 2048])
DEBUG: len train: 8201, len val: 1171, len testN: 2344
Ep 1/20 - Loss: 5.2783 - β: 0.44
Ep 2/20 - Loss: 1.7041 - β: 0.89
Ep 3/20 - Loss: 1.4397 - β: 1.33
Ep 4/20 - Loss: 1.2862 - β: 1.78
Ep 5/20 - Loss: 1.1993 - β: 2.22
Ep 6/20 - Loss: 1.1437 - β: 2.67
Ep 7/20 - Loss: 1.1050 - β: 3.11
Ep 8/20 - Loss: 1.0739 - β: 3.56
Ep 9/20 - Loss: 1.0526 - β: 4.00
Ep 10/20 - Loss: 0.9931 - β: 0.00
Ep 11/20 - Loss: 1.1948 - β: 0.44
Ep 12/20 - Loss: 1.0010 - β: 0.89
Ep 13/20 - Loss: 1.0040 - β: 1.33
Ep 14/20 - Loss: 1.0034 - β: 1.78
Ep 15/20 - Loss: 0.9995 - β: 2.22
Ep 16/20 - Loss: 0.9981 - β: 2.67
Ep 17/20 - Loss: 0.9954 - β: 3.11
Ep 18/20 - Loss: 0.9947 - β: 3.56
Ep 19/20 - Loss: 0.9934 - β: 4.00
Ep 20/20 - Loss: 0.9923 - β: 0.00


In [170]:
# 5) Evaluación: scores, mahala, grid-search
from pprint import pprint
scores_val = compute_anomaly_scores(model, loader_val, device)
mu_norm_val = scores_val['mu']
mbar, inv = fit_mahalanobis(mu_norm_val)
dM_val = mahalanobis_distance(mu_norm_val, mbar, inv)
best = grid_search_alpha(scores_val['mse'], dM_val, loader_val.dataset[:][1])
pprint(best)


{'alpha': 0, 'auc': 0}


c:\ProgramData\anaconda3\envs\TP-final_ML\lib\site-packages\sklearn\metrics\_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
c:\ProgramData\anaconda3\envs\TP-final_ML\lib\site-packages\sklearn\metrics\_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
c:\ProgramData\anaconda3\envs\TP-final_ML\lib\site-packages\sklearn\metrics\_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
c:\ProgramData\anaconda3\envs\TP-final_ML\lib\site-packages\sklearn\metrics\_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
c:\ProgramData\anaconda3\envs\TP-final_ML\lib\site-packages\sklearn\metrics\_ranking.py:379: UndefinedMetricWarning: Only one class is present i

In [171]:
# 6) Detectores no supervisados e inferencia en test
iso = IsolationForest(contamination='auto').fit(mu_val)

ocsvm = OneClassSVM(gamma='auto', nu=0.01).fit(mu_norm_val)

scores_test = compute_anomaly_scores(model, loader_test, device)
mu_test = scores_test['mu']
dM_test = mahalanobis_distance(mu_test, mbar, inv)
comb_test = best['alpha']*scores_test['mse'] + (1-best['alpha'])*dM_test
iso_s = -iso.decision_function(mu_test)
oc_s  = -ocsvm.decision_function(mu_test)

In [172]:
# 7) Balancear para métricas
y_test = np.concatenate([np.zeros(len(ds_test.datasets[0])), np.ones(len(ds_test.datasets[1]))])
idx_n = np.where(y_test==0)[0]
idx_a = np.where(y_test==1)[0]
sel_a = np.random.choice(idx_a, size=len(idx_n), replace=False)
idx = np.concatenate([idx_n, sel_a])
for name, sc in [('ELBO',scores_test['score']),
                    ('Mahala',dM_test),('Combined',comb_test),
                    ('IF',iso_s),('OC',oc_s)]:
    print(f"AUC {name}: {roc_auc_score(y_test[idx], sc[idx]):.4f}")

AUC ELBO: 0.5015
AUC Mahala: 0.2605
AUC Combined: 0.2605
AUC IF: 0.5154
AUC OC: 0.3669


otro

In [173]:
norms = normals
ptb   = anom_ptb
chap  = anom_chap

In [174]:
anos  = np.concatenate([ptb, chap], axis=0) 

In [175]:
# Convert to tensors with proper channel dim
X_norm = torch.tensor(norms[:,1:2,:], dtype=torch.float32)
X_ano  = torch.tensor(anos[:,1:2,:], dtype=torch.float32) 

In [176]:

# Split normals into train/val
n_norm = len(X_norm)
i1 = int(0.6 * n_norm)
i2 = int(0.8 * n_norm)
train_norm = X_norm[:i1]
val_norm   = X_norm[i1:i2]

In [179]:
from pipeline_improved import beta_linear, compute_scores, XGBClassifier

In [ ]:

# Convert to tensors with proper channel dim (Lead II)
# load_sanos() returns shape (n_samples, n_leads, seq_len)
# Select lead II (index 1)
norms_lead = norms[:, 1, :]
anos_lead = anos[:, 1, :]
# Make shape (n_samples, 1, seq_len)
X_norm = torch.tensor(norms_lead, dtype=torch.float32).unsqueeze(1)
X_ano  = torch.tensor(anos_lead, dtype=torch.float32).unsqueeze(1)

# Split normals into train/val
n_norm = len(X_norm)
i1 = int(0.6 * n_norm)
i2 = int(0.8 * n_norm)
train_norm = X_norm[:i1]
val_norm   = X_norm[i1:i2]

# Hyperparameter grid
grid = {
    'latent_dim': [8, 16, 32],
    'lr': [1e-3, 1e-4],
    'n_blocks': [2, 3],
    'beta_fn': [('linear', beta_linear), ('cyclic', beta_cyclic)]
}
results = []

for ld in grid['latent_dim']:
    for lr in grid['lr']:
        for nb in grid['n_blocks']:
            for name, beta_fn in grid['beta_fn']:
                # a) Train VAE on normals only
                train_ds = TensorDataset(train_norm, torch.zeros(len(train_norm)))
                train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
                model = VAE1D(input_ch=1, latent_dim=ld, n_blocks=nb).to(device)
                optimizer = torch.optim.Adam(model.parameters(), lr=lr)
                for epoch in range(10):
                    model.train()
                    for x, _ in train_loader:
                        x = x.to(device)
                        mu, logv = model.encode(x)
                        z = model.reparameterize(mu, logv)
                        rec = model.decode(z)
                        recon_loss = ((rec - x)**2).mean()
                        kl_loss = (-0.5 * (1 + logv - mu.pow(2) - logv.exp()).sum())/x.size(0)
                        beta = beta_fn(epoch, 10)
                        loss = recon_loss + beta * kl_loss
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # b) Prepare validation set: equal normals and anomalies
                N = len(val_norm)
                val_ano = X_ano[:N]
                X_val = torch.cat([val_norm, val_ano], dim=0)
                y_val = np.concatenate([np.zeros(N), np.ones(N)])
                val_ds = TensorDataset(X_val, torch.tensor(y_val, dtype=torch.long))
                val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

                # c) Compute scores on validation
                beta_last = beta_fn(9, 10)
                errs, zs = compute_scores(model, val_loader, device, beta_last)

                # d) Mahalanobis
                mu_bar = zs[:N].mean(axis=0)
                cov = np.cov(zs[:N].T) + 1e-6 * np.eye(ld)
                invcov = np.linalg.inv(cov)
                dM = np.sqrt(((zs - mu_bar) @ invcov * (zs - mu_bar)).sum(axis=1))

                # e) Combined sweep
                best_a, best_auc = 0, 0
                for a in [0, 0.25, 0.5, 0.75, 1]:
                    comb = a * errs + (1 - a) * dM
                    auc = roc_auc_score(y_val, -comb)
                    if auc > best_auc:
                        best_a, best_auc = a, auc

                # f) XGBoost classifier on validation set
                Xf = np.hstack([errs.reshape(-1,1), zs])
                clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
                clf.fit(Xf, y_val)
                probs = clf.predict_proba(Xf)[:,1]
                clf_auc = roc_auc_score(y_val, probs)

                results.append({
                    'latent_dim': ld,
                    'lr': lr,
                    'n_blocks': nb,
                    'beta': name,
                    'best_alpha': best_a,
                    'auc_comb': best_auc,
                    'auc_xgb': clf_auc
                })
# Save sweep results
pd.DataFrame(results).to_csv('hyperparam_sweep_results.csv', index=False)
print("Sweep completo. Resultados en hyperparam_sweep_results.csv")


RuntimeError: Given groups=1, weight of size [32, 1, 19], expected input[32, 0, 2048] to have 1 channels, but got 0 channels instead